# ODBC. Connection to databases 

## Connection to PostgreSQL server with Psycopg2
Installation Psycopg2: pip install psycopg2 (psycopg2==2.7.7)

## Creating the connection to PostgreSQL database

In [105]:
import psycopg2
from psycopg2 import Error
try:
    connection = psycopg2.connect(user = "postgres",
                                  password = "root",
                                  host = "127.0.0.1",
                                  port = "5432",
                                  database = "Interns")
    
    cursor = connection.cursor() #Handles the connection

except (Exception, psycopg2.DatabaseError) as error :
    print ("Error while opening", error)

## Close the connection

In [104]:
#closing database connection.
if(connection):
    cursor.close()
    connection.close()
    print("PostgreSQL connection is closed")

PostgreSQL connection is closed


## Query data

#### Creating a new table

In [106]:
createTable = '''create table InternsInformation(
                    id varchar(15),
                    username varchar(4),
                    name varchar(30),
                    lastname varchar(40),
                    age integer
                );'''

cursor.execute(createTable)
connection.commit()
print("Table created successfully in PostgreSQL ")

Table created successfully in PostgreSQL 


#### Inserting data into a table

In [107]:
# the data set
interns = [
    ('402310453','asge','Andres','Gutierrez',23),
    ('123456790','cope','Carlos','Portuguez',22),
    ('546413431','dima','David','Morales',22),
    ('189146131','waee','William','Espinoza',22),
]
for intern in interns:
    insert = (f"insert into InternsInformation (id,username,name,lastname,age)" 
              f"values (\'{intern[0]}\',\'{intern[1]}\',\'{intern[2]}\',\'{intern[3]}\',{intern[4]});")
    cursor.execute(insert)
    connection.commit()
    count = cursor.rowcount
    print (count, "Record inserted successfully into InternsInformation table")

1 Record inserted successfully into InternsInformation table
1 Record inserted successfully into InternsInformation table
1 Record inserted successfully into InternsInformation table
1 Record inserted successfully into InternsInformation table


#### Select data from a table

In [112]:
# the SQL QUERY
postgreSQL_select_Query = "select * from InternsInformation"

cursor.execute(postgreSQL_select_Query)
print("Selecting rows from InternsInformation table using cursor.fetchall")
records = cursor.fetchall() 

print("Print each row and it's columns values")
for row in records:
   print(row)

# As you can see, it returns a tuple, so we can use all the built-in methods for tuples (e.g. [])

Selecting rows from InternsInformation table using cursor.fetchall
Print each row and it's columns values
('402310453', 'asge', 'Andres', 'Gutierrez', 23)
('123456790', 'cope', 'Carlos', 'Portuguez', 22)
('546413431', 'dima', 'David', 'Morales', 22)
('189146131', 'waee', 'William', 'Espinoza', 22)
('110055252', 'mlsa', 'Melissa', 'Cardenas', 21)


## Lets create a dataframe with Pandas

In [75]:
import pandas as pd

## Executing a stored procedured

In [109]:
# insert_intern procedure perform an insert of a intern into the InternsInformation table
# and prints the inserted value

'''
CREATE OR REPLACE FUNCTION insert_intern (  idp varchar(15),
											usernamep varchar(4),
											namep varchar(30),
											lastnamep varchar(40),
											agep integer) 
 RETURNS TABLE (idr varchar(15),usernamer varchar(4),namer varchar(30),lastnamer varchar(40),ager integer) 
AS $$
BEGIN
	insert into InternsInformation values (idp,usernamep,namep,lastnamep,agep);
	RETURN QUERY 
	SELECT id,username,name,lastname,age FROM InternsInformation WHERE InternsInformation.id = idp ;
END; $$ 
LANGUAGE 'plpgsql';
'''
;

''

#### Calling the procedure

In [111]:
cursor.callproc('insert_intern',('110055252', 'mlsa', 'Melissa', 'Cardenas', 21,))       
row = cursor.fetchone()
while row is not None:
    print(row)
    row = cursor.fetchone()

(5,)


(4,)
